In [1]:
# -*- coding: utf-8 -*-
import torch
import copy
from core.loader import Loader
from models.model_wrapper import ModelWrapper
from core.loader import Loader
import numpy as np

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

2025-05-13 17:18:53.063645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-13 17:18:53.063713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-13 17:18:53.065423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 17:18:53.071312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 17:18:53.786737: W tensorflow/compiler/tf2

2025-05-13 17:18:54.470097: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-05-13 17:18:54.470121: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: poli
2025-05-13 17:18:54.470126: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: poli
/home/poli/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2025-05-13 17:18:54.470166: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostic

### Example for real time classification

In [2]:
from core.validator import load_saved_split, load_train_split
from pipeline import DomainClassifier
import sys

MALICIOUS_LABEL = "phishing"
STAGE = 3
VERIFICATION = True
fp, fn, tp, tn = 0, 0, 0, 0

x_train_additional, y_test_additional = load_saved_split(STAGE, MALICIOUS_LABEL, folder="./data/", verification=VERIFICATION)


DomainClassifier = DomainClassifier(data_sample = x_train_additional, label=MALICIOUS_LABEL)
DomainClassifier.determine_stage(x_train_additional)





for domain, result in zip(x_train_additional, y_test_additional):
    
    
    final_class = DomainClassifier.classify_proba(domain)['final_proba']
    
    print(f"Final Class: {final_class}")
    input()
    
    if result == 1 and final_class > 0.5:
        tp += 1
    elif result == 1 and final_class <= 0.5:
        fn += 1
    elif result == 0 and final_class > 0.5:
        fp += 1
    elif result == 0 and final_class <= 0.5:
        tn += 1
    else:
        print("Error in classification")
        print(domain, result, final_class)
        sys.exit(1)



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Stage determined: 3
Stage: 3
📦 Loading model from models/cnn_stage_3_phishing_v1.1.keras
📦 Loading model from models/svm_stage_3_phishing_v1.1.pkl
📦 Loading model from models/XgBoost_stage_3_phishing_v1.1.xgb
📦 Loading model from models/Lgbm_stage_3_phishing_v1.1.pkl
📦 Loading model from models/feedforward_stage_3_phishing_v1.1.keras
Stage determined: 3
[CNN] X_test reshaped to (1, 14, 14, 1) (side 14, padded by 20)
1/1 [==============================] - 0s 100ms/step
🔄 Detected SVM, using: scalers/phishing_svm_3_scaler.joblib scaler
1/1 [==============================] - 0s 63ms/step
Final Class: 1.0


KeyboardInterrupt: Interrupted by user

## Compute metrics for given stage

In [ ]:
import os
import time
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
)
from core.validator import load_saved_split
from pipeline import DomainClassifier

STAGE = 1
VERIFICATION = True
LABELS = ["phishing", "malware"]
SAVE_PATH = "tex_sources/pipeline_verif.tex"

os.makedirs("tex_sources", exist_ok=True)

results = []

for label in LABELS:
    print(f"\n=== Evaluating {label.upper()} (Stage {STAGE}) ===")

    # Load test data
    x_data, y_data = load_saved_split(STAGE, label, folder="./data/", verification=VERIFICATION)
    x_data = x_data[:10000]
    y_data = y_data[:10000]

    clf = DomainClassifier(data_sample = x_data, label=label)
    clf.determine_stage(x_data)
    

    y_true = []
    y_pred = []
    y_proba = []

    start_time = time.time()

    for x, true_label in zip(x_data, y_data):
        result = clf.classify_proba(x)
        final_proba = result["final_proba"]
        pred_label = 1 if final_proba > 0.5 else 0
        y_true.append(true_label)
        y_pred.append(pred_label)
        y_proba.append(final_proba)

    elapsed = time.time() - start_time
    speed = len(y_true) / elapsed

    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Metrics
    acc = (tp + tn) / (tp + tn + fp + fn)
    prec = tp / (tp + fp) if tp + fp > 0 else 0.0
    recall = tp / (tp + fn) if tp + fn > 0 else 0.0
    f1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0.0
    auc = roc_auc_score(y_true, y_proba)
    
    stage_results = {
        "Stage": STAGE,
        "Label": label,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC": auc,
        "Domains/sec": round(speed, 2),
    }

    results.append(stage_results)
    print(f"Stage results: {stage_results}")
    
    

# Save results to LaTeX
with open(SAVE_PATH, "w") as f:
    for result in results:
        f.write(f"""\\begin{{table}}[H]
\\centering
\\begin{{tabular}}{{|l|c|}}
\\hline
\\textbf{{Metrika}} & \\textbf{{{result['Label'].capitalize()}}} \\\\
\\hline
Přesnost (Accuracy) & \\texttt{{{result['Accuracy']:.4f}}} \\\\
Precision (Přesnost) & \\texttt{{{result['Precision']:.4f}}} \\\\
Recall (Úplnost) & \\texttt{{{result['Recall']:.4f}}} \\\\
F1 Skóre & \\texttt{{{result['F1 Score']:.4f}}} \\\\
ROC AUC & \\texttt{{{result['ROC AUC']:.4f}}} \\\\
\\hline
\\end{{tabular}}
\\caption{{Výsledky klasifikace {result['Label']} domén – verifikační sada}}
\\label{{tab:final_pipeline_ver_{result['Label']}_{result['Stage']}}}
\\end{{table}}

""")

print(f"Saved LaTeX results to: {SAVE_PATH}")



=== Evaluating PHISHING (Stage 1) ===
Stage determined: 1
Stage: 1
📦 Loading model from models/cnn_stage_1_phishing_v1.1.keras
📦 Loading model from models/svm_stage_1_phishing_v1.1.pkl
📦 Loading model from models/XgBoost_stage_1_phishing_v1.1.xgb
📦 Loading model from models/Lgbm_stage_1_phishing_v1.1.pkl
📦 Loading model from models/feedforward_stage_1_phishing_v1.1.keras
Stage determined: 3


ValueError: X has 62 features, but StandardScaler is expecting 176 features as input.